# Algorithme génétique

## Les algorithmes évolutifs

### Algorithmes génétiques

Le Machine Learning peut apparaître complexe de prime abord, et pour cause certains des algorithmes qui constituent cet ensemble sont très loin d’être accessibles.

Ceci étant, il existe un type d’algorithme évolutif plus simple à saisir parmi ceux-ci: les algorithmes génétiques (ou AG).

### Principes

Comme leur nom l'indique, les AGs ont certains points communs avec la génétique en biologie.

Un individu est un ensemble de données concaténées, chacune de ses données représente un gène.

L'ensemble des individus représente la population, tandis que chacune des populations (une par étape de l'algorithme est une génération.

D'une génération à l'autre, l'individu le plus apte à répondre à la problématique est conservé tandis que les autres sont obtenus via le croisement des individus de la génération précédente. Au terme de ces croisements, des modifications peuvent avoir lieu: les mutations.

### Une particularité

L'utilisation d'un AG n'est pas, en soi, très compliqué. Ainsi, nous allons ajouter un peu d'éléments relatifs au Big Data et à ses principes dans ces travaux.

Le principe est finalement simple. Le Big Data, et le traitement de données qui l'accompagne sont définis autour de trois grandes étapes qui sont, pour rappel, le Data Cleaning, le Data Mining et enfin le Data Analysis. A chaque nouvelle étape, un nouveau niveau de détail, plus précis, doit être étudié.

Nous verrons dans le déroulé de ce TD les points importants analogues à ces trois grandes étapes de traitement des données.

## Comment faire?

Au final, les AGs ne sont pas particulièrement compliqués à implémenter, et ce TD vous permettra de créer un exemple simple.

**Attention: Le contenu que vous produirez ici sera réutilisé lors du TD Examen AG, qui sera noté. Veillez à la modularité de votre code.**

**Egalement, pensez à commenter votre code et à le rendre lisible (voir la norme PEP8)**

### Objectif

A la fin de ce TD, votre code sera capable de trouver, de lui-même, une chaîne de caractères définie. Bien sûr, l'utilité est faible, mais il s'agit de pouvoir traiter ensuite toute sorte de données, comme par exemple, utiliser l'AG pour estimer une courbe à partir d'un ensemble de points fournis.

Pour ce TD, le code devra retrouver la chaîne de caractères "Hello World".

### Description

Pour fonctionner, l'AG a besoin:
- d'une fonction d'évaluation permettant d'indiquer la proximité ou l'erreur entre l'individu et la solution optimale. Dans notre cas, la distance entre chaque caractère sera utilisée, la fonction d'évaluation pour une distance optimale vaudra donc 0.
- d'une fonction de croisement qui permet d'obtenir une nouvelle génération à partir d'une génération pré-existante, prenant une partie de deux individus pour en former un nouveau, et ce pour l'ensemble de la population.
- de paramètres de mutation, indiquant les chances pour un nouvel individu d'être altéré, et la force de cette altération.

### Avant le code

In [25]:
# tous les imports de ce TD devront être placés ici
import random as rd
import numpy as np
import os

#Chemin relatif où se trouve le fichier
notebook_path = os.path.abspath("Nicolas_LEGER_MIAGE_SIO_TD2.ipynb")

#Nombre de ligne : 47
def get_result_from_csv(name, csv_file):
    tab_x = []
    tab_y = []
    
    with open(csv_file, "r") as file:
        data = file.readlines()
        #Pour chaque ligne du fichier csv on indique le separateur
        for line in data:
            line = line.split(",")
            #On vérifie le nom de la personne
            if line[1] == name:
                #On ne prend pas en compte les éléments vides ou retour à la ligne
                if '' not in line and '\n' not in line:
                    
                    #On vérifie que les x soit des entiers
                    if float(line[2]).is_integer() == True and float(line[3]).is_integer() == True:
                        if float(line[4]).is_integer() == True and float(line[5]).is_integer() == True:
                            if float(line[6]).is_integer() == True and float(line[7]).is_integer() == True:
                                
                                #On vérifie que les x soient entre -1000 et 1000
                                if -1000 <= int(line[2]) <= 1000 and -1000 <= int(line[3]) <= 1000 and -1000 <= int(line[4]) <= 1000: 
                                    if -1000 <= int(line[5]) <= 1000 and -1000 <= int(line[6]) <= 1000:
                                        
                                        #Le chiffre 0 apparaît trop souvent
                                        #On concidère ça comme une incohérence
                                        #On ne prend pas en compte les lignes qui possèdent 0
                                        if int(line[2]) != 0 and int(line[3]) != 0 and  int(line[4]) != 0:
                                            if int(line[5]) != 0 and int(line[6]) != 0 and int(line[7]) != 0:
                    
                                                #On vérifie la borne de y (entre -5 000 000 et 5 000 000)
                                                if -5000000 <= int(line[7]) <= 5000000:
                                                    
                                                    #On récupère uniquement la parti du y sans le \n
                                                    line[7] = line[7].split("\n")[0]
                                                    
                                                    #Génère une liste des x et l'ajoute dans la liste de tous les x
                                                    tab_x.append([int(line[2]), 
                                                                int(line[3]), 
                                                                int(line[4]), 
                                                                int(line[5]), 
                                                                int(line[6])])
                                                    
                                                    #Ajout du y dans la liste de tous les y
                                                    tab_y.append(int(line[7]))
                                    
    return len(tab_x), tab_x, tab_y
# On obtien le fichier .csv à partir de l'endroit ou se trouve le fichier .ipynb
# Pour ma part les deux sont dans le même dossier
tab_value = get_result_from_csv("Leger", os.path.join(os.path.dirname(notebook_path), "./ExamColl.csv"))

### Création des méthodes

#### Calcul de distance et localisation du meilleur individu

Avant toute chose, il faut réfléchir à la manière de représenter nos données.

Nous traiterons des chaînes de caractères. En Python, une chaîne de caractères fonctionne (presque) exactement comme une liste de caractères, et ces caractères peuvent être remplacés par leur code ASCII, qui est numérique.

Comme dit précédemment, nous allons définir notre individu cible comme la liste des codes ASCII de "H", "e", "l", "l", ... "d", qui aura comme distance 0.

Puisque nous avons les codes ASCII des caractères à disposition, il est simple de trouver, pour chaque indice de la liste, la distance entre le caractère cible et le caractère courant, qui n'est rien de plus qu'une simple différence. De même, la distance globale de la chaîne est une somme des distances.

Une fois les distances d'un ensemble de chaînes obtenues, celles-ci peuvent être ordonnées.

In [ ]:
# la fonction de conversion d'une chaîne de caractères en liste de valeurs ASCII vous est founie
def string_to_int_list(string):
    return [ord(character) for character in list(string)]

##### Exo 1:
- **Créez la méthode permettant de calculer la distance entre un mot et un autre**
- **Vérifiez que la distance avec "Hello World" est correcte pour les mots suivants: "COjsy OfUkp" (105) et "Hemlo Wohld" (11)**

In [14]:
# On a un c
# On le multiplie par tous les X, 
# On additionne les multiplications
# Soustrait à y
def get_coef_for_distance(mon_y, mon_x, mon_c):
    resultat = 0
    for i in range(5):
        resultat = resultat + mon_c[i] * mon_x[i]

    return abs(mon_y - resultat)


def get_distance(mes_y, mes_x, mon_c):
    final_distance = 0
    for i in range(len(mes_y)):
        final_distance = final_distance + get_coef_for_distance(mes_y[i], mes_x[i], mon_c)

    return final_distance

Maintenant qu'il est possible d'attribuer une valeur de distance entre deux mots, nous pouvons ordonner des mots grâce à cette valeur.

##### Exo 2:
- **Créez la méthode permettant de retrouver le mot le plus proche d'une cible et la valeur de cette distance**
- **Testez avec une liste de mots définie par vos soins, et la cible "Hello World"**

In [15]:
# Renvoie le meilleur d'une liste et sa distance avec la target
def get_best(mon_y, mon_x, liste_mon_c):
    liste_distance = []
    
    for liste_nombre in liste_mon_c:
        liste_distance.append(get_distance(mon_y,mon_x,liste_nombre))
        
    min_distance = min(liste_distance)
    mot = liste_mon_c[liste_distance.index(min_distance)]
    return mot, min_distance


#### Création d'une première génération

Bien que nous connaissions le mot cible, nous partirons d'une population constituée d'un ensemble de mots générés aléatoirement.

Gardez à l'esprit que la taille de votre population définira la vitesse d'exécution de votre code. Ainsi, bien qu'un ensemble d'individus important  aura la diversité pour atteindre rapidement (en termes d'itérations) la solution, un ensemble plus restreint passera chacune des itérations rapidement.

##### Exo 3:
- **Créez la méthode d'initialisation d'une liste de mots aléatoires**
- **Utilisez cette méthode pour générer aléatoirement un ensemble de mots, et comparez-les à la cible.**

In [16]:
# Fonction qui renvoie une liste de coefficient c
# prends en parametre le nombre de coefficient  que doit contenir la liste
def word_list_init(nombre_de_mot, nb_facteur):
    liste_mot = []
    end_list = []

    for iteration in range(nombre_de_mot):
        for i in range(nb_facteur):
            liste_mot.append(rd.randint(-1000, 1000))
        end_list.append(liste_mot.copy())
        liste_mot.clear()

    return end_list


#### Passage d'une génération à une autre

Les opérations de base sont maintenant établies, il est temps de rendre notre système évolutif.

D'une génération à l'autre, les individus doivent évoluer. Mais, il n'est pas certain que cette évolution se fasse dans la direction espérée.

La théorie de l'évolution dans le domaine biologique indique que l'espèce la plus adaptée à son environnement survivra plus probablement que d'autres. Dans notre cas, nous pouvons discerner l'individu le plus proche de la solution et le conserver (il "survit"). Ainsi, le meilleur individu d'une génération ne peut pas être moins adapté que celui d'une génération passée (principe de non-régression).

Le reste de la population de la nouvelle génération est produite comme dans le cas de la biologie. Deux parties complémentaires (chromosomes) de deux individus (parents) seront combinées pour obtenir un nouvel individu (enfant).

##### Exo 4:
- **Créez la méthode de transition entre deux générations. Notez que:**
 - **Le meilleur individu (ou les x meilleurs individus) devrait être conservé.**
 - **Les individus restants devraient être un croisement d'individus de la liste des mots précédente (meilleur mot précédent compris).**
 - **La liste de mots résultante devrait être de même longueur que la précédente.**
- **Appliquez la méthode de manière itérative, en indiquant à chaque fois le meilleur élément de la génération et la distance avec la cible.**
- **Au bout d'un certain nombre d'itérations, que se passe-t-il?**

In [18]:
# Prends 2 coeffcient en parametres
# Renvoie un nouveau coefficient en melangeant les 2 coefficient donnees
def crossover_nombre(mon_c1, mon_c2):
    enfant = []
    list_number1_length = len(mon_c1)
    # Il y a 55% de chance de prendre le premier coefficient de la première liste place en parametre
    # Après optimisation c'est ce qui semble le plus rapide
    for i in range(list_number1_length):
        choice = np.random.choice(np.arange(0, 2), 
                                  p=[0.55, 0.45])
        if choice == 0: 
            enfant.append(mon_c1[i])
        else: 
            enfant.append(mon_c2[i])
    return enfant

def new_generation(mon_y, mon_x, liste_de_c):
    new_generation = []
    len_mon_c = len(liste_de_c)
    
    #Ajoute le coefficient qui a la meilleur distance au tableau new_generation
    best_distance = get_best(mon_y, mon_x, liste_de_c)
    best_word = best_distance[0]
    print("best de la generation: ", best_word)
    new_generation.append(best_word)
    liste_de_c.pop(liste_de_c.index(best_word))

    while len(new_generation) != len_mon_c:
        # la variable randome retourne soit 0 soit 1
        # Il y a 80% de chance que le meilleur coefficient de la génération précédente se reproduise avec un mot aléatoir
        # Après optimisation c'est ce qui semble le plus rapide
        random = np.random.choice(np.arange(0, 2), p=[0.8, 0.2])
        if random == 0:
            random_word1 = new_generation[0]
            random_word2 = rd.choice(liste_de_c)
            new_enfant1 = crossover_nombre(random_word1, random_word2)
            new_enfant2 = mutation(mon_y, mon_x, new_enfant1)
            new_generation.append(new_enfant2)
        elif random == 1:
            random_word1 = rd.choice(liste_de_c)
            random_word2 = rd.choice(liste_de_c)
            new_enfant1 = crossover_nombre(random_word1, random_word2)
            new_enfant2 = mutation(mon_y, mon_x, new_enfant1)
            new_generation.append(new_enfant2)

    
    return new_generation

Nous retrouvons, en fin de compte, un ensemble d'individus très similaires. A la différence de l'ensemble initial, où tous les éléments sont générés aléatoirement, ce point des travaux montre une grande défaillance en termes de diversité.

S'il fallait considérer le lien avec les grandes phases du Big Data, nous pourrions imaginer un travail de Data Cleaning trop poussé, retirant tous les éléments aberrants mais aussi malheureusement certains éléments essentiels à la complétude espérée de la donnée (dans notre cas, l'accès à un nombre suffisant de lettres par emplacement).

#### Créer la diversité génétique

Le principe de mutation permet d'ajouter de la diversité lors de l'évolution de l'AG. En effet, avec une faible population, il est presque certain que le croisement normal d'éléments finisse dans une impasse, on parle de minimum local.

Dans notre cas, la notion de minimum local n'a pas véritable lieu d'être. Il nous suffit d'ajouter de nouveaux éléments innovants pour permettre à l'AG de reprendre son évolution.

Cette innovation prend la forme d'une altération d'un (ou plusieurs) caractère(s) d'un individu lors de sa création pour une nouvelle génération. Pour s'assurer que l'on ne dégrade pas le meilleur individu, il est conseillé de lui éviter cette étape.

La mutation est définie suivant deux paramètres principaux. Tout d'abord, sa probabilité, entre 0 et 1, définit sa fréquence. Aussi, sa force définit l'effet de la mutation, et peut prendre n'importe quelle forme, de +1/-1 à une réaffectation aléatoire. Cette mutation peut s'appliquer lettre par lettre, sur le mot entier, ou sur un ensemble de lettres aléatoires.

### Parallèle avec le Big Data

Ici, vous chercherez à optimiser votre algorithme, de sorte qu'il puisse vous fournir une réponse rapidement. Un manque de diversité, ou sa perte trop rapide, pose problème. De même, une avancée trop précise sans "risque" entraîne des longueurs nuisibles à l'efficacité du travail.

Il peut donc être très intéressant de transposer par analogie les thématique du Big Data ici.

#### Data Cleaning

L'ensemble de mots initial est aléatoire, et la probabilité de trouver un mot intéressant est assez faible. De même, le croisement entre ces mots prend plus d'importance, en termes de progression, que la mutation, qui apport un précision ponctuelle.

#### Data Mining

A supposer que l'ensemble des individus est maintenant à la fois restreinte à des mots moins aléatoires tout en conservant une diversité importante, il est maintenant possible de se concentrer sur une base plus limitée de mots et tenter de les croiser en appliquant des mutations de puissance moyenne.

#### Data Analysis

Maintenant que l'ensemble d'individus est dans un espace de recherche très petit, le croisement va commencer à perdre en efficacité (les mots sont presque tous identiques). Il est temps d'éliminer les doublons et de concentrer le travail de précision en ne conservant que la mutation, avec les paramètres les plus faibles possibles.

##### Exo 5:
- **Créez une méthode permettant de définir le procédé de mutation**
- **Utilisez cette méthode pour obtenir la terminaison de votre AG (réduire la distance du meilleur mot à 0)**
- **Votre code est complet, faites varier certains de vos paramètres afin de le rendre plus efficace, si vous le souhaitez**
- **Rendez votre code adaptatif, capable d'acception diverses longueurs de chaînes de caractères.**

In [19]:
# Prend en parametre les différents facteurs et leurs résultats
def mutation(mon_y, mon_x, mon_c ):
    longueur_c = len(mon_c)
    distance = get_distance(mon_y, mon_x, mon_c)
    
    # Pour chaque caractere du mot donne en parametre
    # 1 chance sur 4 d'effectuer la mutation
    for i in range(longueur_c):
        chance = rd.randint(0,3)
        if chance == 0:
            # le poids change en fonction de la distance
            poids_mutation = distance //10000
            random_number = rd.randint(-1 * poids_mutation, poids_mutation)
            
            #On vérifie que les coefficient se situe bien dans les bornes après la mutation
            if -1000 > (random_number + mon_c[i]) > 1000:
                tmp = (random_number + mon_c[i])
                mon_c[i] = tmp % 1000
            else:
                mon_c[i] = (random_number + mon_c[i])
    
    return mon_c

Le bloc suivant vous permet de tester l'exécution de votre code dans les conditions de test finales. Les paramètres des méthodes "word_list_init", "get_distance" et "new_generation" sont à compléter.

La valeur de la variable "test_length" sera modifiée lors de l'évaluation du code.

In [24]:
def algo_genetique(mon_y, mon_x, nb_iteration):
    #Génération de 200 liste composées chacune de 5 coefficients
    list_of_c = word_list_init(200,5)

    print("=== Generation numero :" ,nb_iteration,'===')
    new_gene_list = new_generation(mon_y, mon_x, list_of_c)
    best_word = get_best(mon_y, mon_x, new_gene_list)[0]
    best_distance = get_best(mon_y, mon_x, new_gene_list)[1]
    print("le meilleur mot est :", best_word,"avec une distance de:",best_distance,"\n")
    
    while best_distance != 0:
        nb_iteration += 1
        print("=== Generation numero :" ,nb_iteration,'===')
        new_gene_list = new_generation(mon_y,mon_x, new_gene_list)
        best_word = get_best(mon_y,mon_x, new_gene_list)[0]
        best_distance = get_best(mon_y, mon_x, new_gene_list)[1]
        print("     le meilleur mot est :", best_word,"avec une distance de:",best_distance)
        print("")
    print("\n=============RESULTAT=============")

    return best_word, nb_iteration

print(algo_genetique(tab_value[2], tab_value[1], 0))
# Le résultat obtenu est : 
# =============RESULTAT=============
#([-864, -919, 540, -119, 549], 17)
#Donc les coefficient sont -864, -919, 540, -119, 549
#Et on les a obtenu au bout de 17 iterations

=== Generation numero : 0 ===
best de la generation:  [-868, -976, 353, 233, 603]
le meilleur mot est : [-777, -976, 353, -234, 603] avec une distance de: 4537255 

=== Generation numero : 1 ===
best de la generation:  [-777, -976, 353, -234, 603]
     le meilleur mot est : [-777, -976, 594, -87, 603] avec une distance de: 2719735

=== Generation numero : 2 ===
best de la generation:  [-777, -976, 594, -87, 603]
     le meilleur mot est : [-777, -976, 533, -87, 603] avec une distance de: 2301110

=== Generation numero : 3 ===
best de la generation:  [-777, -976, 533, -87, 603]
     le meilleur mot est : [-863, -919, 533, -87, 603] avec une distance de: 1306155

=== Generation numero : 4 ===
best de la generation:  [-863, -919, 533, -87, 603]
     le meilleur mot est : [-863, -919, 533, -87, 563] avec une distance de: 837667

=== Generation numero : 5 ===
best de la generation:  [-863, -919, 533, -87, 563]
     le meilleur mot est : [-863, -919, 533, -109, 563] avec une distance de: 386

### Adaptation finale

Maintenant que vous avez un code qui termine, assez efficacement, il est temps de trouver une application plus ludique, mais non moins sérieuse, puisqu'elle sera notée.

Ici, l'objectif sera de retrouver non plus une phrase, mais plutôt une "image". Vous trouverez dans le bloc suivant, la nouvelle cible de recherche, qui est un ascii art. Un exemple ests fourni, vous pouvez parfaitement changer cet ascii art selon vos goûts.

Deux nouvelles contraintes s'ajoutent.

La première, il est nécessaire de reconnaître les caractères problématiques (par exemple, les guillemets ou le caractère d'échappement "\") pour assurer leur lecture.

La seconde, il ne faut pas oublier que l'ascii est définie sur deux dimensions. Une méthode d'affichage, qui demande la chaîne à afficher et la longueur d'une ligne de dessin est fournée.

### Travail demandé

Vous pouvez choisir de réutiliser les méthodes existantes pour retrouver l'ascii art que vous aurez choisi (taille minimale acceptée : 3x3).

IMPORTANT:

Ce TD est noté et compte pour un quart de la note TD finale.
L'évaluation sera effectuée par le redémarrage du noyau et l'exécution complète du code. Vérifiez la validité de votre travail avec "Noyau" -> "Redémarrer & tout exécuter". Tout code ne fonctionnant pas en suivant cette procédure vaudra 0.

Le barème est le suivant:
- **L'exécution complète attribuera au plus 12 points.** 2 points sont attribués pour chaque méthode correctement implémentée.
- **Les codes terminant seront mis en compétition** 0 à 6 points seront attribués en fonction du classement.
- **La propreté** (respect du PEP8) **vaudra 2 points.** Un code non propre peut faire perdre jusqu'à 3 points.
- Le code doit respecter le côté aléatoire du sujet. Cela inclut la génération initiale, le croisement et la mutation. Toute méthode parmi les trois indiquées ne respectant pas ce point vaudra 0.

Attention, tout jour de retard pour le rendu de ce travail entraînera une pénalité de 5 points.

Aussi ce code devant être utilisé pour le TD Examen AG, il est conseillé d'y mettre du soin.

Le rendu prend la forme de ce notebook, à envoyer par mail.